In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os

In [2]:
import sklearn
print(sklearn.__version__)

1.5.0


In [3]:
# Inference Pipeline

def inference_pipeline(new_data, load_path):
    # Load the model and scalers
    clf = joblib.load(os.path.join(load_path, 'clf.pkl'))
    scaler = joblib.load(os.path.join(load_path, 'scaler.pkl'))
    label_encoders = joblib.load(os.path.join(load_path, 'label_encoders.pkl'))
    ohe = joblib.load(os.path.join(load_path, 'one_hot_encoder.pkl'))
    feature_importances = pd.read_csv(os.path.join(load_path, 'feature_importances.csv'))
    min_max_scores = np.load(os.path.join(load_path, 'min_max_scores.npz'))
    min_score = min_max_scores['min_score']
    max_score = min_max_scores['max_score']
    # Load nominal features
    with open(os.path.join(load_path, 'nominal_features.txt'), 'r') as f:
        nominal_features = f.read().splitlines()

    # Load columns for reindexing
    with open(os.path.join(load_path, 'dummy_columns.txt'), 'r') as f:
        dummy_columns = f.read().splitlines()

    with open(os.path.join(load_path, 'features_normalize.txt'), 'r') as f:
        features_normalize = f.read().splitlines()
    # Pre-process the new data
    new_df = pd.DataFrame(new_data)
    
    # Handle categorical features
    for col in label_encoders.keys():
        le = label_encoders[col]
        new_df[col + '_Encoded'] = le.transform(new_df[col])
        new_df.drop(col, axis=1, inplace=True)
    
    # One-Hot Encoding for nominal variables
    ohe_features = ohe.transform(new_df[nominal_features])
    ohe_feature_names = ohe.get_feature_names_out(nominal_features)
    new_df_ohe = pd.DataFrame(ohe_features, columns=ohe_feature_names)
    new_df = pd.concat([new_df.drop(nominal_features, axis=1), new_df_ohe], axis=1)

    # Reindex the new DataFrame to ensure it has the same columns as the training data
    new_df = new_df.reindex(columns=dummy_columns, fill_value=0)
    
    # Normalize the new data
    new_df[features_normalize] = scaler.transform(new_df[features_normalize])


    # Predict the impact level
    y_prob_new = clf.predict_proba(new_df)
    y_class = clf.predict(new_df)
    highest_probabilities_new = np.max(y_prob_new, axis=1)
    
    # Calculate the total score based on feature importance and feature points
    def calculate_total_score(row, feature_importances):
        total_score = 0
        for feature, importance in zip(feature_importances['Feature'], feature_importances['Importance']):
            total_score += row[feature] * importance
        return total_score

    new_df['Total_Score'] = new_df.apply(calculate_total_score, axis=1, feature_importances=feature_importances)
    
    # Calculate the weighted score by multiplying the total score by the highest probability
    new_df['Weighted_Score'] = new_df['Total_Score'] * highest_probabilities_new
    new_df['predicted_class'] = y_class
    new_df['predicted_prob'] = highest_probabilities_new
    
    # Normalize the score to the range [100-1000]
    min_range = 100
    max_range = 1000

    new_df['Normalized_Score'] = min_range + ((new_df['Weighted_Score'] - min_score) / (max_score - min_score)) * (max_range - min_range)

    return new_df[['Total_Score', 'predicted_prob','Weighted_Score', 'Normalized_Score','predicted_class']]

In [4]:
load_path = 'C:/Users/hp/Desktop/IOP'

In [5]:
# Example new data for inference
new_data = {
    'Existing_Customers': [2500],
    'Revenue_Loss_Per_Hour_USD': [5000],
    'Combined_Signal_Strength_dBm': [-70],
    'Combined_Coverage_Area_sqkm': [30],
    'Combined_Packet_Loss_Percentage': [2],
    'Combined_Max_Connections': [3000],
    'Infrastructure_Capacity_Mbps': [500],
    'Signal_Strength_dBm': [-75],
    'Coverage_Area_sqkm': [20],
    'Latency_ms': [50],
    'Packet_Loss_Percentage': [1],
    'Max_Connections': [2000],
    'SLA_Hours': [5],
    'Operational_Status': ['Operational'],
    'Backup_Available_Quantitative': [1],
    'Outage_Type': ['Man-made'],
    'Outage_Cause': ['Equipment Failure'],
    'Outage_Duration_Hours': [12],
    'Day_of_Week': ['Monday'],
    'Hour_of_Day': [10],
    'Month': ['6'],  # Ensure this is a string
    'Season': ['2'],  # Ensure this is a string
    'Is_Work_Hours': [1],
    'Is_Weekend': [0]
}

In [6]:
# Run inference pipeline
inference_results = inference_pipeline(new_data,load_path)
print(inference_results)

   Total_Score  predicted_prob  Weighted_Score  Normalized_Score  \
0     0.648485            0.53        0.343697        272.493301   

  predicted_class  
0        Critical  


In [7]:
user_input = pd.read_csv("user_input_data.csv")
user_input

Existing_Customers  Revenue_Loss_Per_Hour_USD  \
0                1000                      100.0   

   Combined_Signal_Strength_dBm  Combined_Coverage_Area_sqkm  \
0                        -100.0                          5.0   

   Combined_Packet_Loss_Percentage  Combined_Max_Connections  \
0                              0.0                       500   

   Infrastructure_Capacity_Mbps  Signal_Strength_dBm  Coverage_Area_sqkm  \
0                          50.0               -100.0                 5.0   

   Latency_ms  ...  Backup_Available_Quantitative  Outage_Type  Outage_Cause  \
0         5.0  ...                              0     Man-made     Vandalism   

  Outage_Duration_Hours  Day_of_Week Hour_of_Day Month  Season Is_Work_Hours  \
0                   0.5       Monday           0     1       1             0   

   Is_Weekend  
0           1  

[1 rows x 24 columns]

In [8]:
min_max_scores = np.load(os.path.join(load_path, 'min_max_scores.npz'))
min_score = min_max_scores['min_score']
max_score = min_max_scores['max_score']
min_score,max_score

(array(0.15294432), array(1.1482154))

In [ ]:
min_range + ((0.0 - min_score) / (max_score - min_score)) * (max_range - min_range)

In [1]:
import pandas as pd

# Define cell towers and their coordinates
data = {
    'Cell Tower': ['Tower 1', 'Tower 2', 'Tower 3', 'Tower 4', 'Tower 5',
                   'Tower 6', 'Tower 7', 'Tower 8', 'Tower 9', 'Tower 10'],
    'Latitude': [39.6837, 39.6879, 39.6882, 39.6905, 39.6954,
                 39.6981, 39.7013, 39.7037, 39.7058, 39.7102],
    'Longitude': [-75.7497, -75.7323, -75.7455, -75.7521, -75.7416,
                  -75.7529, -75.7398, -75.7331, -75.7464, -75.7560]
}

df_towers = pd.DataFrame(data)
df_towers

Cell Tower  Latitude  Longitude
0    Tower 1   39.6837   -75.7497
1    Tower 2   39.6879   -75.7323
2    Tower 3   39.6882   -75.7455
3    Tower 4   39.6905   -75.7521
4    Tower 5   39.6954   -75.7416
5    Tower 6   39.6981   -75.7529
6    Tower 7   39.7013   -75.7398
7    Tower 8   39.7037   -75.7331
8    Tower 9   39.7058   -75.7464
9   Tower 10   39.7102   -75.7560

In [2]:
# Example: Calculate coverage radius for each tower (hypothetical)
# Assuming a fixed radius for coverage, or calculate based on technical specifications
df_towers['Coverage_Radius_km'] = 2.5  # Hypothetical coverage radius in kilometers

# Example: Calculate coverage area (assuming circular coverage)
df_towers['Coverage_Area_sqkm'] = df_towers['Coverage_Radius_km'] ** 2 * 3.1416

# Display the dataframe with calculated coverage areas
print(df_towers)

  Cell Tower  Latitude  Longitude  Coverage_Radius_km  Coverage_Area_sqkm
0    Tower 1   39.6837   -75.7497                 2.5              19.635
1    Tower 2   39.6879   -75.7323                 2.5              19.635
2    Tower 3   39.6882   -75.7455                 2.5              19.635
3    Tower 4   39.6905   -75.7521                 2.5              19.635
4    Tower 5   39.6954   -75.7416                 2.5              19.635
5    Tower 6   39.6981   -75.7529                 2.5              19.635
6    Tower 7   39.7013   -75.7398                 2.5              19.635
7    Tower 8   39.7037   -75.7331                 2.5              19.635
8    Tower 9   39.7058   -75.7464                 2.5              19.635
9   Tower 10   39.7102   -75.7560                 2.5              19.635


In [12]:
import pandas as pd
import numpy as np

# Number of cell towers
num_towers = 25

# Generate mock data
data = {
    'Cell Tower': [f'Tower {i+1}' for i in range(num_towers)],
    'Latitude': np.random.uniform(39.5, 40.0, size=num_towers),
    'Longitude': np.random.uniform(-76.0, -75.5, size=num_towers),
    'Infrastructure_Capacity_Mbps': np.random.randint(80, 150, size=num_towers),
    'Signal_Strength_dBm': np.random.randint(-90, -70, size=num_towers),
    'Coverage_Area_sqkm': np.random.uniform(8.0, 15.0, size=num_towers),
    'Latency_ms': np.random.randint(15, 25, size=num_towers),
    'Packet_Loss_Percentage': np.random.uniform(0.2, 1.0, size=num_towers),
    'Max_Connections': np.random.randint(800, 1500, size=num_towers),
    'Existing_Customers': np.random.randint(1000, 2000, size=num_towers),
    'Revenue_Loss_Per_Hour_USD': np.random.randint(800, 1200, size=num_towers),
    'Overlap_Towers': [np.random.choice([f'Tower {j+1}' for j in range(num_towers)], np.random.randint(1, 5), replace=False).tolist() for _ in range(num_towers)],
    'SLA_Hours': np.random.randint(20, 30, size=num_towers),
    'Operational_Status': np.random.choice(['Operational', 'Maintenance', 'Outage'], size=num_towers),
    'Backup_Available': np.random.choice(['No Backup', 'Partial Backup', 'Full Backup'], size=num_towers),
    'Connections_Handled': np.random.randint(900, 1400, size=num_towers)
}

# Create DataFrame
df_towers = pd.DataFrame(data)

# Map qualitative backup availability to quantitative scale
backup_mapping = {
    'No Backup': 0,
    'Partial Backup': 1,
    'Full Backup': 2
}
df_towers['Backup_Available_Quantitative'] = df_towers['Backup_Available'].map(backup_mapping)

df_towers

Cell Tower   Latitude  Longitude  Infrastructure_Capacity_Mbps  \
0     Tower 1  39.533033 -75.826597                            99   
1     Tower 2  39.765772 -75.557563                           141   
2     Tower 3  39.715586 -75.874300                           140   
3     Tower 4  39.571004 -75.553462                           110   
4     Tower 5  39.631784 -75.894642                           112   
5     Tower 6  39.608117 -75.980035                           141   
6     Tower 7  39.512134 -75.823338                           113   
7     Tower 8  39.504150 -75.727554                           138   
8     Tower 9  39.679791 -75.819143                           142   
9    Tower 10  39.814197 -75.958710                            87   
10   Tower 11  39.713825 -75.956589                           132   
11   Tower 12  39.744470 -75.572284                           124   
12   Tower 13  39.963358 -75.893438                           140   
13   Tower 14  39.914619 -75.892056                           113   
14   Tower 15  39.593794 -75.935516                           114   
15   Tower 16  39.698737 -75.861651                           135   
16   Tower 17  39.817274 -75.559684                            98   
17   Tower 18  39.938771 -75.970689                           110   
18   Tower 19  39.985085 -75.563458                           117   
19   Tower 20  39.908921 -75.923469                            98   
20   Tower 21  39.691708 -75.754217                           114   
21   Tower 22  39.981526 -75.521780                            92   
22   Tower 23  39.673692 -75.588590                           142   
23   Tower 24  39.721350 -75.723825                           129   
24   Tower 25  39.882240 -75.545106                           138   

    Signal_Strength_dBm  Coverage_Area_sqkm  Latency_ms  \
0                   -82            9.186016          21   
1                   -80           14.210932          17   
2                   -75           11.689594          17   
3                   -76           13.959963          23   
4                   -77           14.589072          16   
5                   -89           12.027060          19   
6                   -89           14.327648          23   
7                   -86           11.366358          23   
8                   -83            8.391509          23   
9                   -82           10.654519          24   
10                  -81           11.365869          24   
11                  -90           12.141604          15   
12                  -87           14.140751          16   
13                  -75           13.773943          17   
14                  -82           13.047614          16   
15                  -89           10.047062          21   
16                  -76           14.962674          19   
17                  -73           14.585019          24   
18                  -85           14.898379          15   
19                  -72            8.121532          24   
20                  -80           14.416518          17   
21                  -78           11.796268          23   
22                  -75           14.711433          20   
23                  -78           12.033377          24   
24                  -72           14.330071          15   

    Packet_Loss_Percentage  Max_Connections  Existing_Customers  \
0                 0.752972             1261                1375   
1                 0.797610             1007                1523   
2                 0.867609             1351                1395   
3                 0.573837             1178                1862   
4                 0.577298             1005                1121   
5                 0.563521             1297                1911   
6                 0.572102              954                1153   
7                 0.634569             1405                1111   
8                 0.431294             1025                1006   
9  

In [13]:
# Function to calculate derived attributes based on overlap
def calculate_derived_attributes(row, attribute):
    if len(row['Overlap_Towers']) > 0:
        if attribute == 'Signal_Strength_dBm':
            return df_towers.loc[df_towers['Cell Tower'].isin(row['Overlap_Towers']), 'Signal_Strength_dBm'].mean()
        elif attribute == 'Coverage_Area_sqkm':
            return df_towers.loc[df_towers['Cell Tower'].isin(row['Overlap_Towers']), 'Coverage_Area_sqkm'].sum()
        elif attribute == 'Packet_Loss_Percentage':
            return df_towers.loc[df_towers['Cell Tower'].isin(row['Overlap_Towers']), 'Packet_Loss_Percentage'].mean()
        elif attribute == 'Max_Connections':
            return df_towers.loc[df_towers['Cell Tower'].isin(row['Overlap_Towers']), 'Max_Connections'].max()
    else:
        return np.nan


In [14]:
# Calculate derived attributes
df_towers['Combined_Signal_Strength_dBm'] = df_towers.apply(lambda row: calculate_derived_attributes(row, 'Signal_Strength_dBm'), axis=1)
df_towers['Combined_Coverage_Area_sqkm'] = df_towers.apply(lambda row: calculate_derived_attributes(row, 'Coverage_Area_sqkm'), axis=1)
df_towers['Combined_Packet_Loss_Percentage'] = df_towers.apply(lambda row: calculate_derived_attributes(row, 'Packet_Loss_Percentage'), axis=1)
df_towers['Combined_Max_Connections'] = df_towers.apply(lambda row: calculate_derived_attributes(row, 'Max_Connections'), axis=1)

# Display the DataFrame with derived attributes
df_towers

Cell Tower   Latitude  Longitude  Infrastructure_Capacity_Mbps  \
0     Tower 1  39.533033 -75.826597                            99   
1     Tower 2  39.765772 -75.557563                           141   
2     Tower 3  39.715586 -75.874300                           140   
3     Tower 4  39.571004 -75.553462                           110   
4     Tower 5  39.631784 -75.894642                           112   
5     Tower 6  39.608117 -75.980035                           141   
6     Tower 7  39.512134 -75.823338                           113   
7     Tower 8  39.504150 -75.727554                           138   
8     Tower 9  39.679791 -75.819143                           142   
9    Tower 10  39.814197 -75.958710                            87   
10   Tower 11  39.713825 -75.956589                           132   
11   Tower 12  39.744470 -75.572284                           124   
12   Tower 13  39.963358 -75.893438                           140   
13   Tower 14  39.914619 -75.892056                           113   
14   Tower 15  39.593794 -75.935516                           114   
15   Tower 16  39.698737 -75.861651                           135   
16   Tower 17  39.817274 -75.559684                            98   
17   Tower 18  39.938771 -75.970689                           110   
18   Tower 19  39.985085 -75.563458                           117   
19   Tower 20  39.908921 -75.923469                            98   
20   Tower 21  39.691708 -75.754217                           114   
21   Tower 22  39.981526 -75.521780                            92   
22   Tower 23  39.673692 -75.588590                           142   
23   Tower 24  39.721350 -75.723825                           129   
24   Tower 25  39.882240 -75.545106                           138   

    Signal_Strength_dBm  Coverage_Area_sqkm  Latency_ms  \
0                   -82            9.186016          21   
1                   -80           14.210932          17   
2                   -75           11.689594          17   
3                   -76           13.959963          23   
4                   -77           14.589072          16   
5                   -89           12.027060          19   
6                   -89           14.327648          23   
7                   -86           11.366358          23   
8                   -83            8.391509          23   
9                   -82           10.654519          24   
10                  -81           11.365869          24   
11                  -90           12.141604          15   
12                  -87           14.140751          16   
13                  -75           13.773943          17   
14                  -82           13.047614          16   
15                  -89           10.047062          21   
16                  -76           14.962674          19   
17                  -73           14.585019          24   
18                  -85           14.898379          15   
19                  -72            8.121532          24   
20                  -80           14.416518          17   
21                  -78           11.796268          23   
22                  -75           14.711433          20   
23                  -78           12.033377          24   
24                  -72           14.330071          15   

    Packet_Loss_Percentage  Max_Connections  Existing_Customers  ...  \
0                 0.752972             1261                1375  ...   
1                 0.797610             1007                1523  ...   
2                 0.867609             1351                1395  ...   
3                 0.573837             1178                1862  ...   
4                 0.577298             1005                1121  ...   
5                 0.563521             1297                1911  ...   
6                 0.572102              954                1153  ...   
7                 0.634569             1405                1111  ...   
8                 0.43129

In [15]:
# Define cell towers (using the previously generated tower names)
cell_towers = [f'Tower {i+1}' for i in range(50)]

# Define detailed outage types and their respective probabilities
man_made_causes = ['Vandalism', 'Equipment Failure', 'Human Error']
natural_disaster_causes = ['Storm', 'Earthquake', 'Flood']
infrastructure_deficit_causes = ['Power Outage', 'Network Equipment Failure', 'Backhaul Link Failure']

outage_types = ['Man-made', 'Natural Disaster', 'Infrastructure Deficit']
outage_causes = [man_made_causes, natural_disaster_causes, infrastructure_deficit_causes]
outage_probabilities = [0.4, 0.3, 0.3]  # Adjust probabilities as needed

# Generate mock data for outage scenarios
outage_data = []
for _ in range(50):
    outage_type = np.random.choice(outage_types, p=outage_probabilities)
    outage_cause = np.random.choice(outage_causes[outage_types.index(outage_type)])
    num_towers_affected = np.random.randint(1, 5)  # Randomly select 1 to 4 towers affected
    affected_towers = np.random.choice(cell_towers, num_towers_affected, replace=False)
    start_date = pd.Timestamp.now() - pd.Timedelta(days=np.random.randint(1, 30))
    end_date = start_date + pd.Timedelta(hours=np.random.randint(1, 24))
    outage_data.append({
        'Outage_Type': outage_type,
        'Outage_Cause': outage_cause,
        'Affected_Towers': affected_towers,
        'Start_Date': start_date,
        'End_Date': end_date
    })

# Create DataFrame for outage scenarios
df_outages = pd.DataFrame(outage_data)

df_outages

Outage_Type               Outage_Cause  \
0                 Man-made          Equipment Failure   
1                 Man-made                  Vandalism   
2   Infrastructure Deficit  Network Equipment Failure   
3   Infrastructure Deficit      Backhaul Link Failure   
4   Infrastructure Deficit      Backhaul Link Failure   
5         Natural Disaster                 Earthquake   
6                 Man-made          Equipment Failure   
7         Natural Disaster                      Storm   
8         Natural Disaster                 Earthquake   
9         Natural Disaster                      Storm   
10                Man-made          Equipment Failure   
11                Man-made          Equipment Failure   
12  Infrastructure Deficit  Network Equipment Failure   
13                Man-made                  Vandalism   
14        Natural Disaster                 Earthquake   
15                Man-made          Equipment Failure   
16        Natural Disaster                      Storm   
17        Natural Disaster                      Flood   
18                Man-made          Equipment Failure   
19                Man-made                Human Error   
20  Infrastructure Deficit      Backhaul Link Failure   
21        Natural Disaster                      Flood   
22  Infrastructure Deficit      Backhaul Link Failure   
23        Natural Disaster                      Flood   
24  Infrastructure Deficit               Power Outage   
25                Man-made          Equipment Failure   
26        Natural Disaster                      Flood   
27                Man-made                Human Error   
28                Man-made                Human Error   
29        Natural Disaster                      Storm   
30                Man-made                Human Error   
31        Natural Disaster                      Storm   
32                Man-made                Human Error   
33                Man-made                  Vandalism   
34        Natural Disaster                      Flood   
35                Man-made          Equipment Failure   
36  Infrastructure Deficit               Power Outage   
37                Man-made                  Vandalism   
38  Infrastructure Deficit  Network Equipment Failure   
39                Man-made                Human Error   
40                Man-made          Equipment Failure   
41        Natural Disaster                      Flood   
42        Natural Disaster                      Storm   
43                Man-made                  Vandalism   
44  Infrastructure Deficit  Network Equipment Failure   
45        Natural Disaster                 Earthquake   
46        Natural Disaster                 Earthquake   
47        Natural Disaster                      Storm   
48                Man-made                Human Error   
49        Natural Disaster                      Flood   

                             Affected_Towers                 Start_Date  \
0                                 [Tower 33] 2024-05-22 02:03:15.788809   
1                        [Tower 5, Tower 29] 2024-06-17 02:03:15.832666   
2             [Tower 50, Tower 49, Tower 33] 2024-06-14 02:03:15.832666   
3                        [Tower 4, Tower 38] 2024-05-23 02:03:15.832666   
4    [Tower 50, Tower 10, Tower 43, Tower 1] 2024-06-05 02:03:15.847843   
5              [Tower 50, Tower 28, Tower 6] 2024-06-14 02:03:15.847843   
6    [Tower 29, Tower 11, Tower 1, Tower 27] 2024-05-31 02:03:15.847843   
7                                 [Tower 31] 2024-05-31 02:03:15.847843   
8                                  [Tower 7] 2024-06-01 02:03:15.847843   
9                                 [Tower 15] 2024-05-25 02:03:15.847843   
10                                 [Tower 9] 2024-05-26 02:03:15.847843   
11                                [Tower 18] 2024-05-26 02:03:15.847843   
12            [Tower 11, Tower 26, Tower 14] 2024-05-23 02:03:15.847843   
13                      [Tower 22, Tower 30] 2024-05-31 0

In [17]:
# Calculate duration of outage
df_outages['Outage_Duration_Hours'] = (df_outages['End_Date'] - df_outages['Start_Date']).dt.total_seconds() / 3600

# Extract day of the week
df_outages['Day_of_Week'] = df_outages['Start_Date'].dt.day_name()

# Extract hour of the day
df_outages['Hour_of_Day'] = df_outages['Start_Date'].dt.hour

# Extract month and season
df_outages['Month'] = df_outages['Start_Date'].dt.month
df_outages['Season'] = df_outages['Start_Date'].dt.month % 12 // 3 + 1  # 1: Winter, 2: Spring, 3: Summer, 4: Fall

# Example: Calculate time since last outage
df_outages.sort_values(by='Start_Date', inplace=True)  # Ensure data is sorted by start date
#df_outages['Time_Since_Last_Outage_Hours'] = df_outages.groupby('Affected_Towers')['Start_Date'].diff().dt.total_seconds() / 3600

# Determine work hours vs. non-work hours (assuming 9 AM to 5 PM as work hours)
df_outages['Is_Work_Hours'] = df_outages['Start_Date'].dt.hour.between(9, 17)

# Categorize weekend vs. weekday
df_outages['Is_Weekend'] = df_outages['Start_Date'].dt.weekday >= 5

In [21]:
df_outages.shape

(50, 12)

In [23]:
 df_outages_exploded = df_outages.explode('Affected_Towers')

In [24]:
df_combined = pd.merge(df_towers, df_outages_exploded, left_on='Cell Tower', right_on='Affected_Towers', how='inner')

df_combined

Cell Tower   Latitude  Longitude  Infrastructure_Capacity_Mbps  \
0     Tower 1  39.533033 -75.826597                            99   
1     Tower 1  39.533033 -75.826597                            99   
2     Tower 1  39.533033 -75.826597                            99   
3     Tower 1  39.533033 -75.826597                            99   
4     Tower 3  39.715586 -75.874300                           140   
..        ...        ...        ...                           ...   
58   Tower 25  39.882240 -75.545106                           138   
59   Tower 25  39.882240 -75.545106                           138   
60   Tower 25  39.882240 -75.545106                           138   
61   Tower 25  39.882240 -75.545106                           138   
62   Tower 25  39.882240 -75.545106                           138   

    Signal_Strength_dBm  Coverage_Area_sqkm  Latency_ms  \
0                   -82            9.186016          21   
1                   -82            9.186016          21   
2                   -82            9.186016          21   
3                   -82            9.186016          21   
4                   -75           11.689594          17   
..                  ...                 ...         ...   
58                  -72           14.330071          15   
59                  -72           14.330071          15   
60                  -72           14.330071          15   
61                  -72           14.330071          15   
62                  -72           14.330071          15   

    Packet_Loss_Percentage  Max_Connections  Existing_Customers  ...  \
0                 0.752972             1261                1375  ...   
1                 0.752972             1261                1375  ...   
2                 0.752972             1261                1375  ...   
3                 0.752972             1261                1375  ...   
4                 0.867609             1351                1395  ...   
..                     ...              ...                 ...  ...   
58                0.704764             1253                1995  ...   
59                0.704764             1253                1995  ...   
60                0.704764             1253                1995  ...   
61                0.704764             1253                1995  ...   
62                0.704764             1253                1995  ...   

    Affected_Towers                 Start_Date                   End_Date  \
0           Tower 1 2024-05-26 02:03:15.847843 2024-05-26 15:03:15.847843   
1           Tower 1 2024-05-31 02:03:15.847843 2024-05-31 03:03:15.847843   
2           Tower 1 2024-06-05 02:03:15.847843 2024-06-05 15:03:15.847843   
3           Tower 1 2024-06-18 02:03:15.863451 2024-06-18 20:03:15.863451   
4           Tower 3 2024-06-14 02:03:15.847843 2024-06-14 20:03:15.847843   
..              ...                        ...                        ...   
58         Tower 25 2024-05-26 02:03:15.847843 2024-05-26 12:03:15.847843   
59         Tower 25 2024-06-05 02:03:15.863451 2024-06-05 12:03:15.863451   
60         Tower 25 2024-06-16 02:03:15.847843 2024-06-16 05:03:15.847843   
61         Tower 25 2024-06-16 02:03:15.863451 2024-06-16 14:03:15.863451   
62         Tower 25 2024-06-18 02:03:15.847843 2024-06-18 23:03:15.847843   

   Outage_Duration_Hours Day_of_Week  Hour_of_Day  Month  Season  \
0                   13.0      Sunday            2      5       2   
1                    1.0      Friday            2      5       2   
2                   13.0   Wednesday            2      6       3   
3                   18.0     Tuesday            2      6       3   
4                   18.0      Friday            2      6       3   
..                   ...         ...          ...    ...     ...   
58                  10.0      Sunday            2      5       2   
59                  10.0   Wednesday            2      6       3   
60                   3.0      Sunday            2      6       

In [25]:
df_combined.columns

Index(['Cell Tower', 'Latitude', 'Longitude', 'Infrastructure_Capacity_Mbps',
       'Signal_Strength_dBm', 'Coverage_Area_sqkm', 'Latency_ms',
       'Packet_Loss_Percentage', 'Max_Connections', 'Existing_Customers',
       'Revenue_Loss_Per_Hour_USD', 'Overlap_Towers', 'SLA_Hours',
       'Operational_Status', 'Backup_Available', 'Connections_Handled',
       'Backup_Available_Quantitative', 'Combined_Signal_Strength_dBm',
       'Combined_Coverage_Area_sqkm', 'Combined_Packet_Loss_Percentage',
       'Combined_Max_Connections', 'Outage_Type', 'Outage_Cause',
       'Affected_Towers', 'Start_Date', 'End_Date', 'Outage_Duration_Hours',
       'Day_of_Week', 'Hour_of_Day', 'Month', 'Season', 'Is_Work_Hours',
       'Is_Weekend'],
      dtype='object')

In [8]:
import pandas as pd

# Mock data for cell towers with overlapping coverage and customer metrics
data = {
    'Cell Tower': ['Tower A', 'Tower B', 'Tower C', 'Tower D', 'Tower E'],
    'Latitude': [39.6837, 39.6879, 39.6882, 39.6905, 39.6954],
    'Longitude': [-75.7497, -75.7323, -75.7455, -75.7521, -75.7416],
    'Infrastructure_Capacity_Mbps': [100, 120, 90, 110, 95],
    'Signal_Strength_dBm': [-80, -85, -75, -78, -82],
    'Coverage_Area_sqkm': [10.0, 8.0, 12.0, 9.0, 11.0],
    'Latency_ms': [20, 18, 22, 21, 19],
    'Packet_Loss_Percentage': [0.5, 0.3, 0.7, 0.4, 0.6],
    'Max_Connections': [1000, 1200, 900, 1100, 950],
    'Existing_Customers': [1500, 1800, 1200, 1400, 1600],  # Number of existing customers
    'Overlap_Towers': [['Tower B'], ['Tower A', 'Tower C'], ['Tower B'], ['Tower C'], ['Tower A', 'Tower D']]
}

# Create DataFrame
df_towers = pd.DataFrame(data)

# Display the dataframe
df_towers

Cell Tower  Latitude  Longitude  Infrastructure_Capacity_Mbps  \
0    Tower A   39.6837   -75.7497                           100   
1    Tower B   39.6879   -75.7323                           120   
2    Tower C   39.6882   -75.7455                            90   
3    Tower D   39.6905   -75.7521                           110   
4    Tower E   39.6954   -75.7416                            95   

   Signal_Strength_dBm  Coverage_Area_sqkm  Latency_ms  \
0                  -80                10.0          20   
1                  -85                 8.0          18   
2                  -75                12.0          22   
3                  -78                 9.0          21   
4                  -82                11.0          19   

   Packet_Loss_Percentage  Max_Connections  Existing_Customers  \
0                     0.5             1000                1500   
1                     0.3             1200                1800   
2                     0.7              900                1200   
3                     0.4             1100                1400   
4                     0.6              950                1600   

       Overlap_Towers  
0           [Tower B]  
1  [Tower A, Tower C]  
2           [Tower B]  
3           [Tower C]  
4  [Tower A, Tower D]

In [ ]:
df_towers['Combined_Signal_Strength_dBm'] = df_towers.apply(lambda row: 
    df_towers[df_towers['Cell Tower'].isin(row['Overlap_Towers'])]['Signal_Strength_dBm'].mean(), axis=1)

df_towers['Combined_Coverage_Area_sqkm'] = df_towers.apply(lambda row: 
    df_towers[df_towers['Cell Tower'].isin(row['Overlap_Towers'])]['Coverage_Area_sqkm'].sum(), axis=1)

df_towers['Combined_Packet_Loss_Percentage'] = df_towers.apply(lambda row: 
    df_towers[df_towers['Cell Tower'].isin(row['Overlap_Towers'])]['Packet_Loss_Percentage'].mean(), axis=1)

df_towers['Combined_Max_Connections'] = df_towers.apply(lambda row: 
    df_towers[df_towers['Cell Tower'].isin(row['Overlap_Towers'])]['Max_Connections'].max(), axis=1)

In [27]:
import pandas as pd

# Define the data dictionary
data_dictionary = {
    'Attribute': ['Cell Tower', 'Latitude', 'Longitude', 'Infrastructure_Capacity_Mbps',
                  'Signal_Strength_dBm', 'Coverage_Area_sqkm', 'Latency_ms',
                  'Packet_Loss_Percentage', 'Max_Connections', 'Existing_Customers',
                  'Revenue_Loss_Per_Hour_USD', 'Overlap_Towers', 'SLA_Hours',
                  'Operational_Status', 'Backup_Available', 'Connections_Handled',
                  'Backup_Available_Quantitative', 'Combined_Signal_Strength_dBm',
                  'Combined_Coverage_Area_sqkm', 'Combined_Packet_Loss_Percentage',
                  'Combined_Max_Connections', 'Outage_Type', 'Outage_Cause',
                  'Affected_Towers', 'Start_Date', 'End_Date', 'Outage_Duration_Hours',
                  'Day_of_Week', 'Hour_of_Day', 'Month', 'Season', 'Is_Work_Hours',
                  'Is_Weekend'],
    'Category': ['Operational Parameters', 'Operational Parameters', 'Operational Parameters',
                 'Operational Parameters', 'Operational Parameters', 'Operational Parameters',
                 'Operational Parameters', 'Operational Parameters', 'Operational Parameters',
                 'Operational Parameters', 'Operational Parameters', 'Operational Parameters',
                 'Operational Parameters', 'Operational Parameters', 'Operational Parameters',
                 'Operational Parameters', 'Operational Parameters', 'Operational Parameters',
                 'Operational Parameters', 'Operational Parameters', 'Operational Parameters',
                 'Outage Information', 'Outage Information', 'Outage Information', 'Outage Information',
                 'Outage Information', 'Outage Information', 'Outage Information', 'Outage Information',
                 'Outage Information', 'Outage Information', 'Outage Information', 'Outage Information'],
    'Description': [
        'Identifier for each cell tower',
        'Geographic coordinate of the tower\'s location (decimal degrees)',
        'Geographic coordinate of the tower\'s location (decimal degrees)',
        'Maximum capacity of the tower\'s infrastructure in Mbps',
        'Signal strength measured in dBm',
        'Area covered by the tower in square kilometers',
        'Latency in milliseconds for connections through the tower',
        'Percentage of packet loss experienced through the tower',
        'Maximum number of connections the tower can handle simultaneously',
        'Number of existing customers served by the tower',
        'Estimated revenue loss per hour in USD during an outage',
        'List of towers overlapping with the current tower',
        'Service Level Agreement in hours',
        'Current operational status of the tower',
        'Availability of backup systems at the tower',
        'Number of connections typically handled by the tower',
        'Quantitative representation of backup availability (0 = No Backup, 1 = Partial Backup, 2 = Full Backup)',
        'Mean signal strength considering overlapping towers',
        'Total coverage area considering overlapping towers',
        'Mean packet loss percentage considering overlapping towers',
        'Maximum connections handled considering overlapping towers',
        'Type of outage (e.g., Man-made, Natural Disaster, Infrastructure Deficit)',
        'Specific cause of the outage',
        'List of towers affected by the outage',
        'Start date and time of the outage',
        'End date and time of the outage',
        'Duration of the outage in hours',
        'Day of the week when the outage occurred',
        'Hour of the day when the outage started',
        'Month when the outage occurred',
        'Season of the year when the outage occurred',
        'Boolean indicating if the outage occurred during typical work hours (9 AM to 5 PM)',
        'Boolean indicating if the outage occurred on a weekend'
    ]
}

# Create a DataFrame
df_data_dictionary = pd.DataFrame(data_dictionary)

# Define the file path
file_path = 'data_dictionary.xlsx'

# Save to Excel
df_data_dictionary.to_excel(file_path, index=False)

print(f"Data dictionary saved to {file_path}")


Data dictionary saved to data_dictionary.xlsx


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Define number of records
num_records = 2000

# Generate synthetic data
data = {
    'Cell Tower': [f'Tower {i+1}' for i in range(num_records)],
    'Latitude': np.random.uniform(39.5, 40.0, size=num_records),
    'Longitude': np.random.uniform(-76.0, -75.5, size=num_records),
    'Infrastructure_Capacity_Mbps': np.random.randint(80, 150, size=num_records),
    'Signal_Strength_dBm': np.random.randint(-90, -70, size=num_records),
    'Coverage_Area_sqkm': np.random.uniform(8.0, 15.0, size=num_records),
    'Latency_ms': np.random.randint(15, 25, size=num_records),
    'Packet_Loss_Percentage': np.random.uniform(0.2, 1.0, size=num_records),
    'Max_Connections': np.random.randint(800, 1500, size=num_records),
    'Existing_Customers': np.random.randint(1000, 2000, size=num_records),
    'Revenue_Loss_Per_Hour_USD': np.random.randint(800, 1200, size=num_records),
    'Overlap_Towers': [np.random.choice([f'Tower {j+1}' for j in range(num_records)], np.random.randint(1, 5), replace=False).tolist() for _ in range(num_records)],
    'SLA_Hours': np.random.randint(20, 30, size=num_records),
    'Operational_Status': np.random.choice(['Operational', 'Maintenance', 'Outage'], size=num_records),
    'Backup_Available': np.random.choice(['No Backup', 'Partial Backup', 'Full Backup'], size=num_records),
    'Connections_Handled': np.random.randint(900, 1400, size=num_records),
    'Backup_Available_Quantitative': np.random.randint(0, 3, size=num_records),  # Mapping No Backup: 0, Partial Backup: 1, Full Backup: 2
    'Outage_Type': np.random.choice(['Man-made', 'Natural Disaster', 'Infrastructure Deficit'], size=num_records),
    'Outage_Cause': np.random.choice(['Vandalism', 'Equipment Failure', 'Human Error', 'Storm', 'Earthquake', 'Flood', 'Power Outage', 'Network Equipment Failure', 'Backhaul Link Failure'], size=num_records),
    'Affected_Towers': [np.random.choice([f'Tower {j+1}' for j in range(num_records)], np.random.randint(1, 5), replace=False).tolist() for _ in range(num_records)],
    'Start_Date': [datetime.now() - timedelta(days=np.random.randint(1, 30)) for _ in range(num_records)],
    'End_Date': [datetime.now() - timedelta(days=np.random.randint(1, 30)) + timedelta(hours=np.random.randint(1, 24)) for _ in range(num_records)]
}

# Create DataFrame
df = pd.DataFrame(data)

In [2]:
# Function to calculate derived attributes based on overlap
def calculate_derived_attributes(row, attribute):
    if len(row['Overlap_Towers']) > 0:
        if attribute == 'Combined_Signal_Strength_dBm':
            return df.loc[df['Cell Tower'].isin(row['Overlap_Towers']), 'Signal_Strength_dBm'].mean()
        elif attribute == 'Combined_Coverage_Area_sqkm':
            return df.loc[df['Cell Tower'].isin(row['Overlap_Towers']), 'Coverage_Area_sqkm'].sum()
        elif attribute == 'Combined_Packet_Loss_Percentage':
            return df.loc[df['Cell Tower'].isin(row['Overlap_Towers']), 'Packet_Loss_Percentage'].mean()
        elif attribute == 'Combined_Max_Connections':
            return df.loc[df['Cell Tower'].isin(row['Overlap_Towers']), 'Max_Connections'].max()
    else:
        return np.nan

# Calculate derived attributes
df['Combined_Signal_Strength_dBm'] = df.apply(lambda row: calculate_derived_attributes(row, 'Combined_Signal_Strength_dBm'), axis=1)
df['Combined_Coverage_Area_sqkm'] = df.apply(lambda row: calculate_derived_attributes(row, 'Combined_Coverage_Area_sqkm'), axis=1)
df['Combined_Packet_Loss_Percentage'] = df.apply(lambda row: calculate_derived_attributes(row, 'Combined_Packet_Loss_Percentage'), axis=1)
df['Combined_Max_Connections'] = df.apply(lambda row: calculate_derived_attributes(row, 'Combined_Max_Connections'), axis=1)

# Generate Impact Level based on some logic (example: random assignment)
df['Impact_Level'] = np.random.choice(['Catastrophic', 'Critical', 'Major', 'Minor', 'Information'], size=num_records)


In [3]:
df.head()

Cell Tower   Latitude  Longitude  Infrastructure_Capacity_Mbps  \
0    Tower 1  39.597536 -75.801169                            82   
1    Tower 2  39.792157 -75.815152                            93   
2    Tower 3  39.802354 -75.760976                           145   
3    Tower 4  39.908473 -75.549360                           143   
4    Tower 5  39.850294 -75.905790                           141   

   Signal_Strength_dBm  Coverage_Area_sqkm  Latency_ms  \
0                  -87           11.523691          18   
1                  -90            8.513003          20   
2                  -88            8.100772          15   
3                  -75           11.791819          17   
4                  -71           10.245939          16   

   Packet_Loss_Percentage  Max_Connections  Existing_Customers  ...  \
0                0.392971              911                1408  ...   
1                0.795995              916                1738  ...   
2                0.373045              956                1428  ...   
3                0.238640              801                1547  ...   
4                0.846762             1411                1690  ...   

              Outage_Type           Outage_Cause  \
0  Infrastructure Deficit      Equipment Failure   
1                Man-made  Backhaul Link Failure   
2        Natural Disaster      Equipment Failure   
3  Infrastructure Deficit  Backhaul Link Failure   
4        Natural Disaster            Human Error   

                                  Affected_Towers                 Start_Date  \
0  [Tower 1611, Tower 1850, Tower 504, Tower 262] 2024-06-07 18:35:58.862024   
1             [Tower 186, Tower 1750, Tower 1060] 2024-05-27 18:35:58.862024   
2                                    [Tower 1169] 2024-05-29 18:35:58.862024   
3                                     [Tower 308] 2024-06-04 18:35:58.862024   
4                                    [Tower 1544] 2024-06-08 18:35:58.862024   

                    End_Date  Combined_Signal_Strength_dBm  \
0 2024-05-29 04:35:58.862024                    -81.000000   
1 2024-06-10 23:35:58.862024                    -88.000000   
2 2024-06-02 10:35:58.862024                    -80.333333   
3 2024-06-01 06:35:58.862024                    -79.500000   
4 2024-05-28 22:35:58.862024                    -81.750000   

   Combined_Coverage_Area_sqkm Combined_Packet_Loss_Percentage  \
0                    24.254718                        0.424419   
1                    14.772278                        0.649235   
2                    36.039622                        0.627147   
3                    23.177609                        0.587721   
4                    50.609733                        0.600016   

  Combined_Max_Connections  Impact_Level  
0                     1317   Information  
1                     1201  Catastrophic  
2                     1196  Catastrophic  
3                      997         Major  
4                     1284   Information  

[5 rows x 27 columns]

In [31]:
df.columns

Index(['Cell Tower', 'Latitude', 'Longitude', 'Infrastructure_Capacity_Mbps',
       'Signal_Strength_dBm', 'Coverage_Area_sqkm', 'Latency_ms',
       'Packet_Loss_Percentage', 'Max_Connections', 'Existing_Customers',
       'Revenue_Loss_Per_Hour_USD', 'Overlap_Towers', 'SLA_Hours',
       'Operational_Status', 'Backup_Available', 'Connections_Handled',
       'Backup_Available_Quantitative', 'Outage_Type', 'Outage_Cause',
       'Affected_Towers', 'Start_Date', 'End_Date',
       'Combined_Signal_Strength_dBm', 'Combined_Coverage_Area_sqkm',
       'Combined_Packet_Loss_Percentage', 'Combined_Max_Connections',
       'Impact_Level'],
      dtype='object')

In [32]:
# Save dataset to CSV (optional)
df.to_csv('synthetic_dataset_with_derived.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Assuming df_full_dataset contains your complete dataset with all attributes including 'Impact_Level'

# Define features and target variable
features = ['Cell Tower', 'Latitude', 'Longitude', 'Infrastructure_Capacity_Mbps',
            'Signal_Strength_dBm', 'Coverage_Area_sqkm', 'Latency_ms',
            'Packet_Loss_Percentage', 'Max_Connections', 'Existing_Customers',
            'Revenue_Loss_Per_Hour_USD', 'Overlap_Towers', 'SLA_Hours',
            'Operational_Status', 'Backup_Available', 'Connections_Handled',
            'Backup_Available_Quantitative', 'Outage_Type', 'Outage_Cause',
            'Affected_Towers', 'Start_Date', 'End_Date',
            'Combined_Signal_Strength_dBm', 'Combined_Coverage_Area_sqkm',
            'Combined_Packet_Loss_Percentage', 'Combined_Max_Connections']

target = 'Impact_Level'

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df_full_dataset[features], df_full_dataset[target], test_size=0.2, random_state=42)


In [ ]:
# # Compute combined benefits considering overlaps
# df_towers['Combined_Signal_Strength_dBm'] = df_towers.apply(
#     lambda row: (row['Signal_Strength_dBm'] +
#                  sum(df_towers.loc[df_towers['Cell Tower'].isin(row['Overlap_Towers']), 'Signal_Strength_dBm'])) / (len(row['Overlap_Towers']) + 1),
#     axis=1
# )

# df_towers['Combined_Coverage_Area_sqkm'] = df_towers.apply(
#     lambda row: (row['Coverage_Area_sqkm'] +
#                  sum(df_towers.loc[df_towers['Cell Tower'].isin(row['Overlap_Towers']), 'Coverage_Area_sqkm'])) / (len(row['Overlap_Towers']) + 1),
#     axis=1
# )

# df_towers['Combined_Packet_Loss_Percentage'] = df_towers.apply(
#     lambda row: (row['Packet_Loss_Percentage'] +
#                  sum(df_towers.loc[df_towers['Cell Tower'].isin(row['Overlap_Towers']), 'Packet_Loss_Percentage'])) / (len(row['Overlap_Towers']) + 1),
#     axis=1
# )

# df_towers['Combined_Max_Connections'] = df_towers.apply(
#     lambda row: (row['Max_Connections'] +
#                  sum(df_towers.loc[df_towers['Cell Tower'].isin(row['Overlap_Towers']), 'Max_Connections'])) / (len(row['Overlap_Towers']) + 1),
#     axis=1
# )


# df_towers